In [1]:
# libraries
using Flux              # the julia ml library
using Images            # image processing and machine vision for julia
using MLJ               # make_blobs, rmse, confmat, categorical
using MLDataUtils       # label, nlabel, labelfreq
using MLDatasets        # mnist

using GLM               # (lm works as regression; GLM not OK for categorical outcomes)
using MLJLinearModels   # LogisticClassifier

using LinearAlgebra     # pinv pseudo-inverse matrix
using Metrics           # r2-score
using Random
using StatsBase         # standardize (normalization)
using Distributions

using Plots; gr()
using StatsPlots
using Printf

using CSV
using DataFrames


In [2]:
all_models = models()

186-element Vector{NamedTuple{(:name, :package_name, :is_supervised, :abstract_type, :deep_properties, :docstring, :fit_data_scitype, :hyperparameter_ranges, :hyperparameter_types, :hyperparameters, :implemented_methods, :inverse_transform_scitype, :is_pure_julia, :is_wrapper, :iteration_parameter, :load_path, :package_license, :package_url, :package_uuid, :predict_scitype, :prediction_type, :supports_class_weights, :supports_online, :supports_training_losses, :supports_weights, :transform_scitype, :input_scitype, :target_scitype, :output_scitype), T} where T<:Tuple}:
 (name = ABODDetector, package_name = OutlierDetectionNeighbors, ... )
 (name = ABODDetector, package_name = OutlierDetectionPython, ... )
 (name = AEDetector, package_name = OutlierDetectionNetworks, ... )
 (name = ARDRegressor, package_name = ScikitLearn, ... )
 (name = AdaBoostClassifier, package_name = ScikitLearn, ... )
 (name = AdaBoostRegressor, package_name = ScikitLearn, ... )
 (name = AdaBoostStumpClassifier, pa

In [38]:
model = @load NeuralNetworkClassifier

import MLJFlux ✔

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/ciro/.julia/packages/MLJModels/Ci1zC/src/loading.jl:168


MLJFlux.NeuralNetworkClassifier

In [46]:
models("Logistic")[2]

Classifier corresponding to the loss function ``L(y, Xθ) + λ|θ|₂²/2 + γ|θ|₁`` where `L` is the logistic loss.
→ based on [MLJLinearModels](https://github.com/alan-turing-institute/MLJLinearModels.jl)
→ do `@load LogisticClassifier pkg="MLJLinearModels" to use the model.`
→ do `?LogisticClassifier` for documentation.
→ based on [MLJLinearModels](https://github.com/alan-turing-institute/MLJLinearModels.jl).
→ do `@load LogisticClassifier pkg="MLJLinearModels"` to use the model.
→ do `?LogisticClassifier` for documentation.
(name = "LogisticClassifier",
 package_name = "MLJLinearModels",
 is_supervised = true,
 abstract_type = Probabilistic,
 deep_properties = (),
 docstring =
     """
     Classifier corresponding to the loss function ``L(y, Xθ) + λ|θ|₂²/2 + γ|θ|₁`` where `L` is the logistic loss.
     → based on [MLJLinearModels](https://github.com/alan-turing-institute/MLJLinearModels.jl)
     → do `@load LogisticClassifier pkg="MLJLinearModels" to use the model.`
     → do `?LogisticC